## Installations

PS: This recipe is not working. More info here:
https://github.com/langchain-ai/langchain-weaviate/issues/207

In [ ]:
%pip install -U weaviate-client llama-index llama-index-vector-stores-weaviate llama-index-embeddings-openai llama-index-readers-web

In [1]:
import nest_asyncio
nest_asyncio.apply()


## Connect to Weaviate

In [ ]:
import os
import weaviate
from weaviate import classes as wvc
print(weaviate.__version__)
client = weaviate.connect_to_embedded(
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]
    }
)

In [3]:
# let's print the server version to test the connection

def get_version() -> str:
    response = client.get_meta()
    return response.get("version")

print(get_version())

1.26.6


## Create Schema

In [4]:
client.collections.delete("BlogPost")
collection = client.collections.create(
    name="BlogPost",
    description="Blog Post from Weaviate website",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
            model="text-embedding-3-small"
        )
)

{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"warning","msg":"prop len tracker file /Users/dudanogueira/.local/share/weaviate/blogpost/Bk1GcPd0lOyH/proplengths does not exist, creating new tracker","time":"2024-12-17T16:42:55-03:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-12-17T16:42:55-03:00","wait_for_cache_prefill":false}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"Created shard blogpost_Bk1GcPd0lOyH in 2.695084ms","time":"2024-12-17T16:42:55-03:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":

## Load in Data

In [5]:
from llama_index.readers.web import SimpleWebPageReader

loader = SimpleWebPageReader(html_to_text=True)
blog = loader.load_data(urls=['https://weaviate.io/blog/llamaindex-and-weaviate'])

/Users/dudanogueira/dev/weaviate/lab/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [6]:
from weaviate.util import generate_uuid5
# we need to have uuid in places
# lets generate
for d in blog:
    d.id_ = generate_uuid5(d.id_)

In [7]:
blog[0]

Document(id_='30140a72-a9c9-50df-9201-a40ee5ce0063', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[← Back to Blogs](/blog)\n\nSkip to main content\n\n[![Weaviate](/img/site/weaviate-logo-horizontal-\nlight-1.svg)![Weaviate](/img/site/weaviate-logo-horizontal-\ndark-1.svg)****](/)[.](/developers/weaviate/api/rest)\n\nProduct\n\n  *     * Overview\n    * [Vector Database](/platform)\n    * [Workbench](/workbench)\n    * [Integrations](/product/integrations)\n\n    * Deployment\n    * [Serverless Cloud](/deployment/serverless)\n    * [Enterprise Cloud](/deployment/enterprise-cloud)\n    * [Bring Your Own Cloud](/deployment/byoc)\n    * [Enablement](/deployment/enablement)\n\n    * Overview\n    * [Vector Database](/platform)\n    * [Workbench](/workbench)\n    * [Integrations](/product/integrations)\n    * Deployment\n    * [Serverless Cloud](/deployment/ser

## Construct Vector Store

In [12]:
#let first close the server, and instantiate an async client
client.close()
aclient = weaviate.use_async_with_embedded()

{"action":"restapi_management","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","docker_image_tag":"localhost","level":"info","msg":"Shutting down... ","time":"2024-12-17T16:43:52-03:00"}
{"action":"restapi_management","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","docker_image_tag":"localhost","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-12-17T16:43:52-03:00"}
{"action":"telemetry_push","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"telemetry terminated","payload":"\u0026{MachineID:9755179d-2563-493d-b445-906460fd7276 Type:TERMINATE Version:1.26.6 NumObjects:128 OS:darwin Arch:arm64 UsedModules:[generative-openai text2vec-openai]}","time":"2024-12-17T16:43:53-03:00"}
{"build_git_commit":"ab0312d5d","build_go_ver

{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","index":"Bebidas","level":"info","msg":"reload local index","time":"2024-12-17T16:43:56-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","index":"Podcast","level":"info","msg":"reload local index","time":"2024-12-17T16:43:56-03:00"}
{"build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","index":"VERBA_DOCUMENTS","level":"info","msg":"reload local index","time":"2024-12-17T16:43:56-03:00"}
{"action":"telemetry_push","build_git_commit":"ab0312d5d","build_go_version":"go1.23.1","build_image_tag":"localhost","build_wv_version":"1.26.6","level":"info","msg":"telemetry started","payload":"\u0026{MachineID:a1806579-697f-470c-865c-6e5460c53742 Type:INIT Version:1.26.6 NumObjects:0 OS:darwin Arch:arm64 UsedModules:[generative-openai te

In [8]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.storage_context import StorageContext
import os
import openai

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(
    embed_batch_size=10,
    model="text-embedding-3-small"
)
openai.api_key = os.environ["OPENAI_APIKEY"]
Settings.embed_model = embed_model

# construct vector store
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="BlogPost"
)

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# set up the index
index = VectorStoreIndex(blog, storage_context=storage_context)

query_engine = index.as_query_engine()

## We can instantiate our Vector Store Index at any time


In [13]:
vector_store = WeaviateVectorStore(
    weaviate_client=aclient, index_name="BlogPost"
)

loaded_index = VectorStoreIndex.from_vector_store(vector_store)

query_engine = loaded_index.as_query_engine(use_async=False)

## Set up Sub Question Query Engine

In [14]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine_tools = [
    QueryEngineTool(
        query_engine = query_engine,
        metadata = ToolMetadata(name='BlogPost', description='Blog post about the integration of LlamaIndex and Weaviate')
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

/Users/dudanogueira/dev/weaviate/lab/.venv/lib/python3.12/site-packages/pydantic/main.py:1355: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/Users/dudanogueira/dev/weaviate/lab/.venv/lib/python3.12/site-packages/llama_index/program/openai/base.py:143: DeprecationWarning: Call to deprecated class OpenAIPydanticProgram. (Please use `FunctionCallingProgram` instead.)
  return cls(


## Query Time

In [15]:
response = await query_engine.query('How does LlamaIndex help data indexing in Weaviate?')

/Users/dudanogueira/dev/weaviate/lab/.venv/lib/python3.12/site-packages/pydantic/main.py:1179: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(
/Users/dudanogueira/dev/weaviate/lab/.venv/lib/python3.12/site-packages/pydantic/main.py:1187: PydanticDeprecatedSince20: `load_str_bytes` is deprecated. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  obj = parse.load_str_bytes(


Generated 3 sub questions.
[BlogPost] Q: What is LlamaIndex?
[BlogPost] Q: What is Weaviate?
[BlogPost] Q: How does LlamaIndex integrate with Weaviate?


WeaviateClosedClientError: The `WeaviateClient` is closed. Run `client.connect()` to (re)connect!

In [ ]:
print(response)